In [1]:
from itertools import chain
import json
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import pandas as pd
from h3 import h3

In [2]:
project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

data_raw = data_dir/'raw'
data_int = data_dir/'interim'
data_out = data_dir/'processed'

gdb_raw = data_raw/'raw.gdb'
gdb_int = data_int/'interim.gdb'
gdb_out = data_out/'processed.gdb'

# import the project package from the project package path
sys.path.append(str(project_parent/'src'))
import h3_arcgis

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# Start Analysis Prototyping

In [7]:
raw_trips_csv = data_raw/'raw_trips.csv'
raw_df = pd.read_csv(raw_trips_csv, index_col=0).reset_index(drop=True)

raw_df.SHAPE = raw_df.SHAPE.swifter.apply(lambda geom: Geometry(eval(geom)))
raw_df.spatial.set_geometry('SHAPE')

raw_df.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\h3-py\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,travel_distance_miles,travel_time_minutes,SHAPE,store_locnum
0,13.261921,25.033333,"{'x': -123.026415, 'y': 44.87728, 'spatialRefe...",413963145
1,8.260506,12.466667,"{'x': -123.150157, 'y': 44.933511, 'spatialRef...",244210043
2,2.013863,8.750000,"{'x': -123.312642, 'y': 44.929472, 'spatialRef...",244210043
3,0.547428,7.516667,"{'x': -123.30801, 'y': 44.9299, 'spatialRefere...",244210043
4,0.133595,3.100000,"{'x': -123.315767, 'y': 44.922983, 'spatialRef...",244210043


In [25]:
h3_res = 6

raw_df['h3_id'] = raw_df.SHAPE.swifter.apply(lambda geom: h3.geo_to_h3(geom.centroid[1], geom.centroid[0], h3_res))

raw_df.head()

,travel_distance_miles,travel_time_minutes,SHAPE,store_locnum,h3_id
0,13.261921,25.033333,"{'x': -123.026415, 'y': 44.87728, 'spatialRefe...",413963145,8628f42e7ffffff
1,8.260506,12.466667,"{'x': -123.150157, 'y': 44.933511, 'spatialRef...",244210043,8628f55b7ffffff
2,2.013863,8.750000,"{'x': -123.312642, 'y': 44.929472, 'spatialRef...",244210043,8628f5517ffffff
3,0.547428,7.516667,"{'x': -123.30801, 'y': 44.9299, 'spatialRefere...",244210043,8628f5517ffffff
4,0.133595,3.100000,"{'x': -123.315767, 'y': 44.922983, 'spatialRef...",244210043,8628f5517ffffff


In [26]:
df = raw_df.h3_id.value_counts().to_frame().reset_index()
df.columns = ['h3_id', 'count']

df.head()

,h3_id,count
0,8628f0007ffffff,262296
1,8628f0027ffffff,121275
2,8628f0037ffffff,83914
3,8628f002fffffff,58735
4,8628f0157ffffff,53811


In [27]:
df_geom = h3_arcgis.get_h3_spatially_enabled_dataframe(df, 'h3_id')

df_geom.head()

,h3_id,count,SHAPE
0,8628f0007ffffff,262296,"{'rings': [[[-122.65751269299994, 45.515817929..."
1,8628f0027ffffff,121275,"{'rings': [[[-122.68027255599998, 45.462072752..."
2,8628f0037ffffff,83914,"{'rings': [[[-122.60160280699995, 45.479434998..."
3,8628f002fffffff,58735,"{'rings': [[[-122.73617703499997, 45.498419726..."
4,8628f0157ffffff,53811,"{'rings': [[[-122.75887378399995, 45.444660979..."


In [28]:
df_geom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1283 entries, 0 to 1282
Data columns (total 3 columns):
h3_id    1283 non-null object
count    1283 non-null int64
SHAPE    1283 non-null geometry
dtypes: geometry(1), int64(1), object(1)
memory usage: 30.2+ KB


In [30]:
from arcgis.gis import GIS
df_geom.spatial.plot(gis=GIS())

MapView(layout=Layout(height='400px', width='100%'))

In [1]:
fnl_df = h3_arcgis.get_nonoverlapping_h3_hexbins_for_points(raw_df, h3_max=10)
fnl_df.head()

NameError: name 'h3_arcgis' is not defined

In [5]:
fnl_df.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))